# MNIST 手寫數字辨識器訓練
使用 VGG16 與批次正規化 (Batch Normalization) 進行訓練。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import os

## 設定設備

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'使用設備: {device}')

## 資料轉換與下載

In [ ]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 下載 MNIST 資料集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

## 建立 DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000, shuffle=False)

## 定義 VGG16 模型並加入 Batch Normalization

In [ ]:
model = models.vgg16_bn(pretrained=False)
# 調整第一層以接受單通道圖片
model.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)
# 調整輸出層為 10 類
model.classifier[6] = nn.Linear(4096, 10)
model = model.to(device)

## 定義損失函數和優化器

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 訓練參數初始化

In [ ]:
epochs = 30
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
best_val_accuracy = 0.0
best_model_path = 'best_model.pth'

## 訓練與驗證迴圈

In [ ]:
for epoch in range(1, epochs + 1):
    # 訓練階段
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * data.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
    
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    # 驗證階段
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            val_running_loss += loss.item() * data.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()
    
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_acc = val_correct / val_total
    val_losses.append(val_epoch_loss)
    val_accuracies.append(val_epoch_acc)
    
    print(f'Epoch {epoch}/{epochs} | '
          f'Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f} | '
          f'Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f}')
    
    # 儲存最佳模型
    if val_epoch_acc > best_val_accuracy:
        best_val_accuracy = val_epoch_acc
        torch.save(model.state_dict(), best_model_path)

## 繪製損失圖

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(1, epochs +1), train_losses, label='訓練損失')
plt.plot(range(1, epochs +1), val_losses, label='驗證損失')
plt.xlabel('Epoch')
plt.ylabel('損失')
plt.title('訓練與驗證損失')
plt.legend()
plt.savefig('loss.png')
plt.close()

## 繪製準確度圖

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(1, epochs +1), train_accuracies, label='訓練準確度')
plt.plot(range(1, epochs +1), val_accuracies, label='驗證準確度')
plt.xlabel('Epoch')
plt.ylabel('準確度')
plt.title('訓練與驗證準確度')
plt.legend()
plt.savefig('accuracy.png')
plt.close()